In [1]:
from langchain.tools import BaseTool
import requests
from typing import Any
import os
from dotenv import load_dotenv

load_dotenv()

class Weather(BaseTool):
    name: str="weather"
    description: str="根据城市获取天气数据"
    
    def __init__(self):
        super().__init__()
        
    def get_weather(self, location):
        api_key = os.getenv("WEATHER_API_KEY")
        url = f"https://api.seniverse.com/v3/weather/now.json?key={api_key}&location={location}&language=zh-Hans&unit=c"
        response=requests.get(url)
        print(location)
        if response.status_code==200:
            data=response.json()
            weather={
                'description':data['results'][0]['now']['text'],
                'temperature':data['results'][0]['now']['temperature']
            }
            return weather
        else:
            raise Exception(f"失败接收天气信息：{response.status_code}")
      
    def _run(self,para):
        print(para)
        return self.get_weather(para)

weather_tool=Weather()
weather_tool.run({"para":"北京"})

北京
北京


{'description': '晴', 'temperature': '0'}

In [2]:
tools=[weather_tool]

In [3]:
from langchain import hub

prompt=hub.pull("hwchase17/xml-agent-convo")
prompt

c:\Users\86151\anaconda3\envs\langchain_env\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tools'], input_types={}, partial_variables={'chat_history': ''}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'xml-agent-convo', 'lc_hub_commit_hash': '00f6b7470fa25a24eef6e4e3c1e44ba07189f3e91c4d987223ad232490673be8'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tools'], input_types={}, partial_variables={}, template="You are a helpful assistant. Help the user answer any questions.\n\nYou have access to the following tools:\n\n{tools}\n\nIn order to use a tool, you can use <tool></tool> and <tool_input></tool_input> tags. You will then get back a response in the form <observation></observation>\nFor example, if you have a tool called 'search' that could run a google search, in order to search for the weather in SF you would respond:\n\n<tool>search</tool><tool_input>weather in SF</tool_input>\n<observation>64 degrees</observation>\n\n

In [4]:
from langchain_openai import OpenAI,ChatOpenAI

model=ChatOpenAI(
  model="qwen2.5-coder-1.5b-instruct",
  openai_api_key="EMPTY",
  base_url="http://127.0.0.1:1234/v1",
  temperature=0.3
)

In [6]:
from langchain.agents import AgentExecutor,create_xml_agent

llm=model

agent=create_xml_agent(llm,tools,prompt)

In [7]:
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [8]:
agent_executor.invoke({"input":"今天上海的天气"})



> Entering new AgentExecutor chain...
<tool>weather</tool><tool_input>上海上海
上海
{'description': '晴', 'temperature': '7'}<final_answer>The weather in Shanghai is sunny with a temperature of 7 degrees.</final_answer>

> Finished chain.


{'input': '今天上海的天气',
 'output': 'The weather in Shanghai is sunny with a temperature of 7 degrees.'}

JSON结构智能体

In [9]:
from langchain.agents import AgentExecutor,create_json_chat_agent
from langchain import hub

prompt=hub.pull("hwchase17/react-chat-json")
prompt

c:\Users\86151\anaconda3\envs\langchain_env\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.System

In [10]:
agent=create_json_chat_agent(llm=llm,prompt=prompt,tools=tools)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_messages(x['intermediate_steps'], template_tool_response=template_tool_response))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], t

In [11]:
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True，handle_parsing_errors="True")

In [12]:
agent_executor.invoke({"input":"今天杭州的天气"})



> Entering new AgentExecutor chain...
```json
{
    "action": "weather",
    "action_input": "杭州"
}
```杭州
杭州
{'description': '晴', 'temperature': '8'}```json
{
    "action": "Final Answer",
    "action_input": "晴"
}
```

> Finished chain.


{'input': '今天杭州的天气', 'output': '晴'}